# Hyperspectral Band Selection

In this notebook, we'll demonstrate how to use our Inter-band redundancy method (IBRA) and the [Greedy Spectral Selection](https://www.mdpi.com/2072-4292/13/18/3649) (GSS) for hyperspectral band selection. IBRA can also be used in combination with Principal Component Analysis (PCA) and Partial Least Squares (PLS) for dimensionality reduction.

## Installation

Execute `!pip install git+https://github.com/NISL-MSU/HSI-BandSelection`



In [1]:
!pip install -q git+https://github.com/NISL-MSU/HSI-BandSelection
import warnings
warnings.filterwarnings("ignore")

## Load your data

You can bring your own HSI classification dataset. Format the input data as a set of image data cubes of shape $(N, w, h, b)$, where $N$ is the number of data cubes, $w$ and $h$ are the width and the height of the cubes, and $b$ is the number of spectral bands. You could use the `createImageCubes` method, provided [here](https://github.com/NISL-MSU/HSI-BandSelection/blob/master/src/HSIBandSelection/readSAT.py#L47), as a reference to format your data.

In this example, we will load the Indian Pines dataset, which is an image with shape `(145, 145, 200)`.

In [2]:
from HSIBandSelection.readSAT import loadata, createImageCubes
X, Y = loadata(name='IP')
print('Initial image shape: ' + str(X.shape))

X, Y = createImageCubes(X, Y, window=5)
print('Processed dataset shape: ' + str(X.shape))

Initial image shape: (145, 145, 200)
Processed dataset shape: (10249, 5, 5, 200)


In this case, we loaded a HS image saved in our package. It doesn't matter where you bring the data from, you only need to provide the $X$ (input data) and $Y$ (target labels) matrices. In addition, assign your dataset a name; otherwise, it will be called `temp`. With these three elements, we create a data object:

In [3]:
from HSIBandSelection.utils import Dataset
dataset = Dataset(train_x=X, train_y=Y, name='IP')

## Execute the Band Selection / Dimensionality Reduction Algorithm

We'll use the `SelectBands` class. **Parameters**:

*   `dataset`: utils.Dataset object
*   `method`: Method name. Options: 'IBRA', 'GSS' (IBRA+GSS), 'PCA' (IBRA+PCA), and 'PLS' (IBRA+PLS)
*   `classifier`: Classifier type. Options: 'CNN' (if data is 2D), 'ANN', 'RF', 'SVM'. *Default:* 'CNN'
*   `nbands`: How many spectral bands you want to select or reduce to. *Default:* 5
*   `transform`: If True, the final selected bands will suffer a Gaussian transformation to simulate being a multispectral band. *Default:* False
*   `average`: If True, average consecutive bands to reduce the initial total # of bands to half. *Default:* False
*   `epochs`: Number of iterations used to train the NN models. *Default:* 150
*   `batch_size`: Batch size used to train the NN models. *Default:* 128
*   `scratch`: If True, execute the IBRA process from scratch and replace previously saved results. *Default:* True












In [4]:
from HSIBandSelection.SelectBands import SelectBands
selector = SelectBands(dataset=dataset, method='GSS', nbands=5)

From the SelectBands class, we call the `run_selection` method. **Parameters**:

*   `init_vf`: Initial Variance Inflation Factor threshold (used for IBRA). *Default: 12*
*   `final_vf`: Final Variance Inflation Factor threshold (used for IBRA). *Default: 5*



**Return**:

If the selected method is IBRA:

*   `VIF_best`: The VIF threshold at which the best results were obtained
*   `IBRA_best`: The best pre-selected bands using Iner-band redundancy
*   `stats_best`: The best performance metric values obtained after 5x2 CV using the selected bands

If the selected method is GSS:

*   `VIF_best`: The VIF threshold at which the best results were obtained
*   `IBRA_best`: The best pre-selected bands using Iner-band redundancy
*   `GSS_best`: The best combination of bands obtained using GSS
*   `stats_best`: The best performance metric values obtained after 5x2 CV using the selected bands

If the selected method is PCA or PLS:

*   `VIF_best`: The VIF threshold at which the best results were obtained
*   `IBRA_best`: The best pre-selected bands using Iner-band redundancy
*   `reduced_dataset`: The reduced dataset after applying PCA or PLS to the pre-selected bands
*   `stats_best`: The best performance metric values obtained after 5x2 CV using the reduced bands

**IMPORTANT**: This code is implemented using Pytorch. If you're running this on Google Colab, change the runtime type to GPU to accelerate the training process!

In [5]:
VIF_best, IBRA_best, GSS_best, stats_best =selector.run_selection(init_vf=11, final_vf=9)

*************************************
Testing VIF threshold: 11
*************************************
Selecting bands:  [0, 7, 11, 15, 17, 20, 26, 34, 37, 39, 47, 56, 58, 60, 67, 74, 78, 89, 99, 104, 109, 125, 142, 144, 146, 148, 150, 169, 191, 198]
Executing IBRA + GSS (Greddy Spectral Selection)
	Analyzing candidate combination 1. 5x2 CV using bands: [17, 20, 26, 37, 47]


100%|██████████| 10/10 [03:02<00:00, 18.22s/it]


	Mean F1: 0.9532722950991902
			Multicolinearity analysis. Variance Inflation Factor of band 17 is 11.77
			Multicolinearity analysis. Variance Inflation Factor of band 20 is 13.91
			Multicolinearity analysis. Variance Inflation Factor of band 26 is 14.69
			Multicolinearity analysis. Variance Inflation Factor of band 37 is 62.25
			Multicolinearity analysis. Variance Inflation Factor of band 47 is 67.33
	Analyzing candidate combination 2. 5x2 CV using bands: [17, 20, 26, 34, 37]


100%|██████████| 10/10 [02:59<00:00, 17.91s/it]


	Mean F1: 0.9625449649975509
	Best selection so far: [17, 20, 26, 34, 37]with an F1 score of 0.9625449649975509
			Multicolinearity analysis. Variance Inflation Factor of band 17 is 12.89
			Multicolinearity analysis. Variance Inflation Factor of band 20 is 13.77
			Multicolinearity analysis. Variance Inflation Factor of band 26 is 14.11
			Multicolinearity analysis. Variance Inflation Factor of band 34 is 7.73
			Multicolinearity analysis. Variance Inflation Factor of band 37 is 1.94
	Analyzing candidate combination 3. 5x2 CV using bands: [15, 17, 20, 34, 37]


...

	Mean F1: 0.9353896605084507
	Best selection so far: [17, 25, 34, 46, 67]with an F1 score of 0.9755117725351754
			Multicolinearity analysis. Variance Inflation Factor of band 34 is 3.34
			Multicolinearity analysis. Variance Inflation Factor of band 56 is 1.64
			Multicolinearity analysis. Variance Inflation Factor of band 60 is 1.44
			Multicolinearity analysis. Variance Inflation Factor of band 86 is 1.51
			Multicolinearity analysis. Variance Inflation Factor of band 173 is 3.99
	Analyzing candidate combination 17. 5x2 CV using bands: [34, 56, 60, 86, 169]


100%|██████████| 10/10 [03:01<00:00, 18.15s/it]


	Mean F1: 0.9294671472103456
	Best selection so far: [17, 25, 34, 46, 67]with an F1 score of 0.9755117725351754
			Multicolinearity analysis. Variance Inflation Factor of band 34 is 3.37
			Multicolinearity analysis. Variance Inflation Factor of band 56 is 1.66
			Multicolinearity analysis. Variance Inflation Factor of band 60 is 1.44
			Multicolinearity analysis. Variance Inflation Factor of band 86 is 1.51
			Multicolinearity analysis. Variance Inflation Factor of band 169 is 4.04
	Analyzing candidate combination 18. 5x2 CV using bands: [0, 34, 56, 60, 86]


100%|██████████| 10/10 [02:59<00:00, 17.99s/it]


	Mean F1: 0.9151186857838709
Selecting bands:  [17, 25, 34, 46, 67]

 Training a model using 5x2 CV using the final selected or reduced bands...
Mean F1 score: 97.328508768817

The best F1 performance was achieved using a VIF = 10
The best band combination obtained using GSS was [11, 25, 34, 39, 67]


In [6]:
print('The best metrics were obtained using a VIF value of {}'.format(VIF_best))
print('The pre-selected bands obtained by IBRA wew {}'.format(IBRA_best))
print('The pre-selected bands obtained by IBRA+GSS wew {}'.format(GSS_best))
print('The best classification metrics were as follows:')
print(stats_best)

The best metrics were obtained using a VIF value of 10
The pre-selected bands obtained by IBRA wew [0, 7, 11, 15, 17, 25, 34, 37, 39, 44, 47, 56, 58, 60, 67, 74, 78, 86, 93, 99, 104, 109, 125, 142, 144, 146, 148, 150, 169, 191, 199]
The pre-selected bands obtained by IBRA+GSS wew [11, 25, 34, 39, 67]
The best classification metrics were as follows:
Stats(mean_accuracy=97.90243902439025, std_accuracy=0.25474439254362624, mean_precision=98.2594345663942, std_precision=0.29439238546148844, mean_recall=97.63255042170843, std_recall=0.727851940440311, mean_f1=97.89117044278429, std_f1=0.41396655152957146)
